#### ... (Опис проекту, далі імплементація) 

### Інсталяція необхідних бібліотек

У нашому проекті ми використовуємо бібліотеки:
- neat (https://neat-python.readthedocs.io/en/latest/) - для генетичного алгоритму(основна бібліотека в нашому проєкті)
- pygame (https://www.pygame.org/docs/) - для візуалізації результатів
- perlin_noise (https://pypi.org/project/perlin-noise/) - для генерації нашого ландшафту

In [1]:
import math
import random

from functools import cache
from collections import namedtuple

import neat
import pygame

from perlin_noise import PerlinNoise

pygame 2.4.0 (SDL 2.26.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Створюємо серидовище

Наше серидовище - це ліс у якому живе багато травин.

In [2]:
"""
file: forest.py
"""

class ForestMap:
    Rect = namedtuple("Rect", ["x", "y", "width", "height", "center"])
    def __init__(self, num_organisms: int = 10, screen_width: int = 800, screen_height: int = 600):
        self.num_organisms = num_organisms
        self.screen_width = screen_width
        self.screen_height = screen_height

        self.screen = None
        self._pygame_init()

        self.organisms = list()
        self.picture = None
        self.road = list()

    def _pygame_init(self):
        pygame.init()
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption("Random Forest Map Generator")

    def generate_map(self):
        """
        Generate a random forest.
        """
        noise = PerlinNoise(octaves=6, seed=random.randint(0, 100000))
        xpix, ypix = self.screen_width, self.screen_height
        pic = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]

        self.picture = pic

    def generate_organisms(self):
        """
        Generate organisms on the map.
        """
        def make_tree(x, y, size):
            tree_x, tree_y = x, y
            tree_rect = pygame.Rect(tree_x - size // 2, tree_y - size // 2, size, size)

            tree_center_x, tree_center_y = tree_rect.center
            tree_rect = self.Rect(tree_center_x, tree_center_y, size, size, (tree_x, tree_y))
            self.organisms.append((tree_rect, (random.randint(200, 255), 0, 0)))

        margin = 10
        for i in range(0, len(self.picture) - margin, margin):
            for j in range(0, len(self.picture[i]) - margin, margin):
                chance = random.random()
                if self.picture[i][j]>=0.2 and chance < 0.3:
                    make_tree(j, i, 4)
                elif self.picture[i][j]>=0.09 and chance < 0.2:
                    make_tree(j, i, 3)
                elif self.picture[i][j]>=0.009 and chance < 0.1:
                    make_tree(j, i, 3)
                elif self.picture[i][j]>=-0.3 and chance < 0.05:
                    make_tree(j, i, 3)

        self.organisms = tuple(self.organisms)

    def _update_picture(self):
        """
        Update the picture of the forest.
        """
        screen = self.screen
        for i, row in enumerate(self.picture):
            for j, column in enumerate(row):
                if column>=0.2:
                    # There will be generated animals
                    pygame.draw.rect(screen, (80, 80, 80), pygame.Rect(j, i, 1, 1))             
                elif column>=0.09:
                    pygame.draw.rect(screen, (30, 90, 30), pygame.Rect(j, i, 1, 1))
                elif column >=0.009:
                    pygame.draw.rect(screen, (10, 100, 10), pygame.Rect(j, i, 1, 1))
                elif column >=0.002:
                    pygame.draw.rect(screen, (100, 150, 0), pygame.Rect(j, i, 1, 1))
                elif column >=-0.06:
                    pygame.draw.rect(screen, (30, 190, 0), pygame.Rect(j, i, 1, 1))
                elif column >=-0.02:
                    pygame.draw.rect(screen, (40, 200, 0), pygame.Rect(j, i, 1, 1))
                elif column >=-0.3:
                    pygame.draw.rect(screen, (10, 210, 0), pygame.Rect(j, i, 1, 1))
                elif column >=-0.8 and column <-0.3:
                    pygame.draw.rect(screen, (0, 0, 200), pygame.Rect(j, i, 1, 1))

    def _update_organisms(self):
        """
        Update the organisms positions in the forest.
        """
        for tree, color in self.organisms:
            pygame.draw.rect(self.screen, color, (tree.x, tree.y, tree.width, tree.height))

    def _update_path(self):
        """
        Draw a path on the screen.
        """
        assert self.screen is not None, "Must call pygame_init() before drawing a path."

        for i in range(len(self.road) - 1):
            pygame.draw.line(self.screen, (255, 255, 255), self.road[i], self.road[i + 1], 2)

    def run(self):
        """
        Run the simulation.
        """
        assert self.screen is not None, "Must call pygame_init() before running the simulation."
        pygame.display.update()

        # Wait for the user to close the window
        running = True
        while running:
            self.screen.fill((0, 0, 0))

            self._update_picture()
            self._update_organisms()
            self._update_path()

            pygame.time.delay(1000)
            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

        pygame.quit()


### Імплементація алгоритму для побудови дороги

#### Імплементація батьківського класу

In [3]:
"""
file: builder.py
"""

class NeatForestRoadBuilder:
    """
    A class that uses NEAT to build a road through a forest.
    """
    def __init__(self, forest_map: ForestMap, num_generations: int = 100, config_path: str = "config-feedforward.txt"):
        self.forest_map = forest_map

        self.forest_organisms: list = forest_map.organisms
        self.screen_width = forest_map.screen_width
        self.screen_height = forest_map.screen_height

        self.road_smoothness = 250
        self.winner = None

        self.num_generations = num_generations
        self.config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                                  neat.DefaultSpeciesSet, neat.DefaultStagnation,
                                  config_path)

    def _build_path(self, net):
        ...

    @staticmethod
    def distance(point1, point2):
        """
        Calculate the distance between two points.
        """
        return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) **2)

    @staticmethod
    @cache
    def nearest_organism_cached(point, organisms, n_first=5):
        """
        Find the nearest organism to a point.
        """
        return sorted(organisms, key=lambda x: __class__.distance(x[0].center, point))[:n_first]

    def is_valid(self, point):
        """
        Check if a point is valid.
        """
        if point[0] < 0 or point[1] < 0 \
            or point[0] >= self.screen_width or point[1] >= self.screen_height:
            return False

        return True

    def eval_genome(self, genome, config):
        """
        Evaluate the fitness of a genome.
        """
        ...

    def eval_genomes(self, genomes, config):
        """
        Evaluate the fitness of a list of genomes.
        """
        for _, genome in genomes:
            genome.fitness = self.eval_genome(genome, config)

    def _find_winner(self):
        pop = neat.Population(self.config)
        winner = pop.run(self.eval_genomes, self.num_generations)
        return winner

    def build(self):
        """
        Build a road through the forest.
        """
        self.winner = self._find_winner() if self.winner is None else self.winner
        net = neat.nn.FeedForwardNetwork.create(self.winner, self.config)

        self.forest_map.road = self._build_path(net)


#### Дочірній клас, який імлементовує в собі побудову дороги та оцінювання

In [4]:
"""
file: builder_feedforward.py
"""

class NeatForestRoadBuilderFeedForward(NeatForestRoadBuilder):
    """
    A class that uses NEAT to build a road through a forest.
    """
    def _build_path(self, net: neat.nn.FeedForwardNetwork):
        """
        'net' instance builds a road through the forest.
        """
        path = [(0, 0), (3, 3)]

        for _ in range(1, self.road_smoothness):
            if self.distance(path[-1], (self.screen_width, self.screen_height)) < 30:
                break

            delta_dest = (
                (self.screen_width - path[-1][0]) / 1,
                (self.screen_height - path[-1][1]) / 1
            )

            current_pos = (
                path[-1][0] / 1,
                path[-1][1] / 1
            )

            nearest = self.nearest_organism_cached(path[-1], self.forest_organisms, 5)
            nearest = filter(lambda x: self.distance(path[-1], x[0].center) < 30, nearest)

            delta_nearest = []
            for point in nearest:
                delta_nearest.append((point[0].center[0] - path[-1][0]) / 1)
                delta_nearest.append((point[0].center[1] - path[-1][1]) / 1)

            odd_point = [current_pos[0] - 30, current_pos[1] - 30]
            inputs = list(delta_dest) + list(current_pos) + list(delta_nearest) + odd_point * int((10 - len(delta_nearest)) / 2)

            step = 8
            output = net.activate(inputs)

            output = list(map(lambda x: 2 * (x - 0.5), output))

            path.append((
                path[-1][0] + output[0] * step,
                path[-1][1] + output[1] * step
            ))

        return path

    def eval_genome(self, genome, config):
        """
        Evaluate the fitness of a genome.
        """
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        path = self._build_path(net)

        fitness = 0

        for point in path:
            if not self.is_valid(point):
                continue

            nearest = self.nearest_organism_cached(point, self.forest_organisms, 5)
            nearest = tuple(filter(lambda x: self.distance(point, x[0].center) < 30, nearest))

            delta_fitness = 1
            for organism in nearest:
                delta_fitness *= self.distance(point, organism[0].center)

            delta_fitness = delta_fitness ** (1.0 / max(len(nearest), 1)) * 0.033 + 1
            fitness += 30 * delta_fitness

        path_length = sum(self.distance(path[i], path[i + 1]) for i in range(len(path) - 1))
        length_prop = 1 + math.fabs(path_length - self.distance((0, 0), (self.screen_height, self.screen_width)))

        delta_dest = self.distance(path[-1], (self.screen_width, self.screen_height))
        return fitness * (1 / (1 + delta_dest * 0.1)) ** 0.8 * (1 / length_prop ** 0.8)


#### Давайте запустимо нашу програму

Спобуємо навчити нашу мережу будувати дорогу в лісі. Встановимо розімір карти, кількість травин. Важливим параметром є кількість епох навчання, яка визначає скільки разів ми будемо навчати нашу мережу. Чим більше епох, тим краще буде результат, але навчання займе більше часу.

In [ ]:
if __name__ == "__main__":
    forest = ForestMap(
        num_organisms = 900,
        screen_width = 600,
        screen_height = 400,
    )

    # Prepare the forest and create organisms.
    forest.generate_map()
    forest.generate_organisms()

    # Create a builder instance. Let's train it for 70 generations.
    builder = NeatForestRoadBuilderFeedForward(forest, 70)
    builder.build()

    forest.run()
